In [1]:
import pandas as pd
import numpy as np
import re
import string
import nltk
nltk.download('wordnet')
from nltk.corpus import stopwords
import gensim
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import word_tokenize
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Tejas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
data  = pd.read_csv(r'C:\Users\Tejas\Desktop\Capstone\Datasets\davidson2017.csv')
df = data[['tweet','class']]

df['tweet_lower']=df['tweet'].apply(lambda x: x.lower())
df['tweet_lower']=df['tweet_lower'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
df['tweet_lower']=df['tweet_lower'].apply(lambda x: re.sub(' +',' ',x))

C:\Users\Tejas\AppData\Local\Temp/ipykernel_8600/456453841.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet_lower']=df['tweet'].apply(lambda x: x.lower())
C:\Users\Tejas\AppData\Local\Temp/ipykernel_8600/456453841.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet_lower']=df['tweet_lower'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
C:\Users\Tejas\AppData\Local\Temp/ipykernel_8600/456453841.py:6: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [3]:
def get_lda_objects(text):
    nltk.download('stopwords')    
    stop=set(stopwords.words('english'))

    
    def _preprocess_text(text):
        corpus=[]
        stem=PorterStemmer()
        lem=WordNetLemmatizer()
        for news in text:
            words=[w for w in word_tokenize(news) if (w not in stop)]

            words=[lem.lemmatize(w) for w in words if len(w)>2]

            corpus.append(words)
        return corpus
    
    corpus=_preprocess_text(text)
    
    dic=gensim.corpora.Dictionary(corpus)
    bow_corpus = [dic.doc2bow(doc) for doc in corpus]
    
    lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 3, 
                                   id2word = dic,                                    
                                   passes = 10,
                                   workers = 2)
    
    return lda_model, bow_corpus, dic

def plot_lda_vis(lda_model, bow_corpus, dic):
    pyLDAvis.enable_notebook()
    vis = gensimvis.prepare(lda_model, bow_corpus, dic)
    return vis

In [4]:
lda_model, bow_corpus, dic = get_lda_objects(df['tweet_lower'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tejas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
lda_model.show_topics()

[(0,
  '0.105*"bitch" + 0.015*"like" + 0.014*"nigga" + 0.011*"fuck" + 0.011*"as" + 0.010*"dont" + 0.009*"get" + 0.009*"shit" + 0.007*"got" + 0.007*"aint"'),
 (1,
  '0.060*"hoe" + 0.020*"pussy" + 0.013*"trash" + 0.009*"nigga" + 0.008*"got" + 0.007*"dont" + 0.007*"aint" + 0.006*"get" + 0.006*"like" + 0.006*"yall"'),
 (2,
  '0.013*"like" + 0.011*"pussy" + 0.007*"faggot" + 0.005*"trash" + 0.005*"youre" + 0.005*"cunt" + 0.004*"fucking" + 0.004*"look" + 0.003*"charlie" + 0.003*"never"')]

In [6]:
plot_lda_vis(lda_model, bow_corpus, dic)

c:\Users\Tejas\Desktop\Capstone\hate-speech-project\hsp\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.124298  0.099285       1        1  45.968341
1      0.038791 -0.141337       2        1  29.540882
2     -0.163089  0.042052       3        1  24.490777, topic_info=          Term          Freq         Total Category  logprob  loglift
22       bitch  10065.000000  10065.000000  Default  30.0000  30.0000
18         hoe   3677.000000   3677.000000  Default  29.0000  29.0000
122      pussy   1794.000000   1794.000000  Default  28.0000  28.0000
9        trash   1083.000000   1083.000000  Default  27.0000  27.0000
88       nigga   1938.000000   1938.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
2532     today     90.512853    171.837848   Topic3  -6.3313   0.7658
198       call    108.546068    322.223825   Topic3  -6.1496   0.3188
1499  retarded     94.604983    218.766395   Topic3  -6.2871   0.5686
91        love    108.516690    585.045324   Topic3  -6.1499  -0.2779
107       dont    115.065826   1516.021820   Topic3  -6.0913  -1.1715

[232 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
11        2  0.993188      1st
142       1  0.622878     aint
142       2  0.377104     aint
6643      2  0.994912    album
1         1  0.515539      amp
...     ...       ...      ...
833       3  0.517885   yellow
149       1  0.439051    youre
149       2  0.006042    youre
149       3  0.553849    youre
2590      3  0.989650  youtube

[344 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])